In [ ]:
#https://selenium-python.readthedocs.io/. Documentation

from selenium import webdriver #Import the Selenium Webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

#Import Pandas
import pandas as pd    
options = Options()
options.headless = True # Make sure that chrome doesn't automatically opens up on it's own.
options.add_argument("--window-size=1920,1200")

#Download the latest chrome-driver from this link (https://sites.google.com/a/chromium.org/chromedriver/home). 
#Make sure that you have an update version of chrome installed on your computer

DRIVER_PATH = '/Users/frankburg/Documents/chromedriver-2'

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

In [ ]:
#Read the the mined_data excel sheet and the unmined data that has been filled up with smiles.

mined_sheet= pd.read_excel('/Users/frankburg/Documents/drugs/mined.xlsx')
unmined_sheet= pd.read_excel('/Users/frankburg/Documents/drugs/unmined.xlsx')

whole_data=pd.concat([mined_sheet,unmined_sheet],ignore_index=True) #Concatenate the two   
whole_data.sort_values(['id'], inplace=True) #Sort using Id                           

smiles= whole_data['SMILES'] #Get only the SMILES fields
                            
chemical_name=whole_data['chemical name'] #Grab only the chemical name column
active_ingred = whole_data['Active Ingredient'] 
drug_name = whole_data['Drug Name'] #Drug name

In [ ]:
def find_CAS(smiles):
    driver.get("https://www.ncbi.nlm.nih.gov/")

    #Search the StdInChIKey on NCBI 
    search_bar = driver.find_element_by_id('term')
    search_bar.clear()
    search_bar.send_keys(smiles)

    driver.find_element_by_id('search').click()
    driver.implicitly_wait(5)
    
    driver.find_element_by_xpath('//*[@id="results-grid"]/div[12]/section/ul/li[2]/a').click()
    driver.implicitly_wait(5)

    #MW = driver.find_element_by_xpath('//*[@id="Computed-Properties"]/div[2]/div/div[1]/table/tbody/tr[1]/td[2]').text

    #Click the href to go to the next page -> PubChem
    driver.find_element_by_xpath('//*[@id="collection-results-container"]/div/div/div[2]/ul/li/div/div/div[1]/div[2]/div[1]/a').click()

    driver.implicitly_wait(5)
    MW = driver.find_element_by_xpath('//*[@id="Computed-Properties"]/div[2]/div/div[1]/table/tbody/tr[1]/td[2]').text

    CAS = driver.find_element_by_xpath('//*[@id="CAS"]/div[2]/div[1]/p').text

    LogP = driver.find_element_by_xpath('//*[@id="Computed-Properties"]/div[2]/div/div[1]/table/tbody/tr[2]/td[2]').text

    Surface_Area=driver.find_element_by_xpath('//*[@id="Computed-Properties"]/div[2]/div/div[1]/table/tbody/tr[8]/td[2]').text

    Complexity =driver.find_element_by_xpath('//*[@id="Computed-Properties"]/div[2]/div/div[1]/table/tbody/tr[11]/td[2]').text
    return(CAS,MW.split(' ')[0],LogP,Surface_Area.split(' ')[0],Complexity)

In [ ]:
#find_CAS('CC=1C=CC(=NC1)C1=NC=C(C=C1)C')

In [ ]:
mined_data=[]
unmined_data=[]
for i,smiles_value in enumerate(smiles[:]):
    try:
        CAS_list = find_CAS(smiles_value) #Call the function find_CAS
        CAS_LIST =list(CAS_list) #Convert the output int list
        whole_list = [i,drug_name[i],active_ingred[i],chemical_name[i],smiles_value] + CAS_LIST #concatenate the whole entire list.
        mined_data.append(whole_list)
        
    except Exception as e:
        unmined_data.append([i, drug_name[i],active_ingred[i],chemical_name[i],smiles_value])        



In [ ]:
#Convert to Pandas DataFrame
mined_df = pd.DataFrame(mined_data,columns=['id','Drug Name','Active Ingredient','chemical name', 'SMILES', 'CAS No','Molecular Weight','LogP','Surface Area','Complexity'])

unmined_df = pd.DataFrame(unmined_data,columns=['id','Drug Name','Active Ingredient','chemical name','SMILES'])

In [ ]:
# Export to Excel
a=mined_df.to_excel('/Users/frankburg/Documents/drugs/CAS_mined.xlsx') # I save in the folder I created called drugs with the name CAS_mined.xlsx. Create on your computer as well.
b=unmined_df.to_excel('/Users/frankburg/Documents/drugs/CAS_unmined.xlsx') 

### CONCLUSION

##### 1. Make sure for your CAS_mined excel sheet exported is accurate and right.
##### 2. For the CAS_Unmined data check that all the SMILES are okay. If yes then you have to manually search Google to complete the other especially the CAS NO, Molecular Weight and LogP.
#### 3. Then merge the two excel sheets ==> The CAS_mined excel sheet and the CAS_Unmined excel that you have manually updated using the script in the cell below this conclusion.
##### 4. You will upload this merged data after naming it ==> Original Datasheet name + _session1 eg. **datasheet_jkov_session1.xlsx** to a Google Drive (https://drive.google.com/drive/folders/1vEf-mRdgXbQ1yA9FCR4NPsv23FIrPlPd?usp=sharing). Create a folder inside this Google Drive with your Team name as the name of the folder and upload the excel datasheet into it.  

In [ ]:
#Combining two excel sheet

mined100=pd.read_excel('/Users/frankburg/Documents/drugs/CAS_mined100.xlsx') # Change to your own path
mined200=pd.read_excel('/Users/frankburg/Documents/drugs/CAS_mined200.xlsx') # Change to your own path
mined300=pd.read_excel('/Users/frankburg/Documents/drugs/CAS_mined300.xlsx') # Change to your own path



unmined100=pd.read_excel('/Users/frankburg/Documents/drugs/CAS_unmined100.xlsx')
unmined200=pd.read_excel('/Users/frankburg/Documents/drugs/CAS_unmined200.xlsx')

combined=pd.concat([mined100,mined200,mined300,unmined100,unmined200],ignore_index=True)
combined.sort_values(['id'], inplace=True)

c=combined.to_excel('/Users/frankburg/Documents/drugs/datasheet_jkov_session1.xlsx') # Change path appropriately